<a href="https://colab.research.google.com/github/Malmasre/Arabic-ACD/blob/main/8_Student_Teacher_Voting_Annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Teacher And Student Ensemble Annotation
---
-**We have two models trained on two different datasets**

*Teacher Model  -> SemEval Dataset------------------------ Student Model -> SemEval + Teacher Annotations*

-**As we noticed That Ensembling the predictions had Increased the score based on this we can ensemble the annotations rather than use one model for this task.**

In [ ]:
!pip install -U -q plotly
!pip install -q pyyaml==5.4.1
!pip install -q ktrain
!pip install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1
!pip install -q git+https://github.com/amaiya/stellargraph@no_tf_dep_082

     |████████████████████████████████| 27.7 MB 1.3 MB/s 
     |████████████████████████████████| 636 kB 6.8 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.1 MB/s 
     |████████████████████████████████| 981 kB 86.8 MB/s 
     |████████████████████████████████| 263 kB 77.8 MB/s 
     |████████████████████████████████| 2.8 MB 63.3 MB/s 
     |████████████████████████████████| 1.2 MB 58.3 MB/s 
     |████████████████████████████████| 468 kB 67.3 MB/s 
     |████████████████████████████████| 895 kB 68.8 MB/s 
     |████████████████████████████████| 3.3 MB 74.8 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
     |████████████████████████

In [ ]:
#allows us to create the Data and Figure objects
from plotly.graph_objs import *
#plotly.plotly pushes your charts to the cloud  
# import chart_studio.plotly as py
# import plotly.figure_factory as ff
import plotly.express as px
# import chart_studio.plotly as py

import matplotlib.pyplot as plt
%matplotlib inline

# work with cufflinks offline and set its theme
import plotly.io as pio
pio.templates.default = "plotly_white"


import pandas as pd
import numpy as np

# from utils import *

import os
import json
import re
import sys

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
import ktrain
from ktrain import text

In [ ]:
def plot_conf_matrix(conf_matrix_array, class_names, title='', normalized = False, colorscale='GnBu'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`
    """

    z_vals = conf_matrix_array[::-1]
    
    title = 'Confusion Matrix<br>{0}'.format(title)
    
    if normalized:
        z_vals =  np.round(z_vals.astype('float')/z_vals.sum(axis=1)[:, np.newaxis], 3)
        title = 'Normalized ' + title
        
    fig = ff.create_annotated_heatmap(x = class_names,
                                      y = class_names[::-1],
                                      z = z_vals, colorscale=colorscale, showscale=True)
    
    
    
    fig.layout.update(title = title,
                      width=700, height=500,
                      yaxis = dict(title = 'True', side = 'left'),
                      xaxis = dict(title = 'Predicted', side = 'bottom'))
    return fig

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.metrics import confusion_matrix as cm, classification_report as cr

metrics_map = {
    'Accuracy': accuracy_score,
    'Precision': precision_score,
    'Recall': recall_score,
    'F1-score': f1_score,
    'Logloss': log_loss
}

In [ ]:
def train_val_test_split(X, y, test_size, val_size, random_state, stratify):
    '''split a multi-labeled data into train, validation, and test sets using'''
    
    X_train, X_val_test, y_train, y_val_test = train_test_split(X, y,
                                                                test_size=val_size+test_size,
                                                                random_state=random_state,
                                                                stratify=stratify)
    
    
    
    X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test,
                                                    test_size=0.5,
                                                    random_state=random_state,
                                                    stratify=y_val_test)
    
    
    
    return X_train, X_val, X_test, y_train, y_val, y_test
    

In [ ]:
import tensorflow as tf
print(tf.__version__)
# # print(tf.test.gpu_device_name())
# # print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.8.0


In [ ]:
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Adam

In [ ]:
def print_full(x):
    pd.set_option('display.max_colwidth', None)
    return x

def reset():
    pd.reset_option('display.max_colwidth')

## Reading The Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PROJECT_PATH = '/content/drive/My Drive/projects/zeroshot-ar/'
DATA_PATH= '/content/drive/My Drive/projects/zeroshot-ar/data/'
HARD_PATH = os.path.join(DATA_PATH, 'HARD/clean-unbalanced-sentences.csv')
SEMEVAL_PATH = os.path.join(DATA_PATH, 'semEval/semeval_clean.json')
PRETRAINED_MODEL_PATH='/content/drive/My Drive/projects/zeroshot-ar/models/bert-multilabel-semEval'
MODEL_PATH = '/content/drive/My Drive/projects/zeroshot-ar/models/bert-multilabel-semEval-and-hard'
RES_PATH = os.path.join(PROJECT_PATH, 'result')


if not os.path.isdir(MODEL_PATH):
  os.makedirs(MODEL_PATH)

if not os.path.isdir(PROJECT_PATH):
  os.makedirs(PROJECT_PATH)

In [ ]:
HARD_PATH

'/content/drive/My Drive/projects/zeroshot-ar/data/HARD/clean-unbalanced-sentences.csv'

In [ ]:
df = pd.read_csv(HARD_PATH)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079341 entries, 0 to 1079340
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   sentence        1079341 non-null  object
 1   review_index    1079341 non-null  int64 
 2   rating          1079341 non-null  int64 
 3   sentence_lemma  1075836 non-null  object
dtypes: int64(2), object(2)
memory usage: 32.9+ MB


In [ ]:
df = df.dropna(subset=['sentence_lemma'])
len(df)

1075836

In [ ]:
SEMEVAL_PATH

'/content/drive/My Drive/projects/zeroshot-ar/data/semEval/semeval_clean.json'

In [ ]:
sem_eval = pd.read_json(SEMEVAL_PATH)

In [ ]:
sem_eval['word_count'] = sem_eval['text_lemma'].apply(lambda x: len(x.split()))
df['word_count'] = df['sentence_lemma'].apply(lambda x: len(x.split()))

In [ ]:
pd.concat([sem_eval[['word_count']].rename(columns={'word_count': 'semeval_word_count'}), 
           df[['word_count']].rename(columns={'word_count': 'hard_word_count'})], axis=1).describe(percentiles=[.25, .5, .75, 0.90, 0.95, 0.99])

,semeval_word_count,hard_word_count
count,6029.000000,1.075836e+06
mean,20.496931,8.078171e+00
std,13.847659,1.200333e+01
min,1.000000,1.000000e+00
25%,10.000000,2.000000e+00
50%,18.000000,4.000000e+00
75%,27.000000,9.000000e+00
90%,39.000000,1.900000e+01
95%,50.000000,2.900000e+01
99%,65.000000,5.700000e+01


In [ ]:
df = df[df['word_count'] <= 65]
len(df)

1068744

In [ ]:
(
    sem_eval['word_count']
    #  .apply(lambda x: len(x.split()))
     .pipe(px.histogram, nbins=50, title='Distribution of Reviews Length')
)

In [ ]:
(
    df['sentence_lemma']
     .dropna()
     .apply(lambda x: len(x.split()))
     .pipe(px.histogram, nbins=50, title='Distribution of Reviews Length')
)

In [ ]:
df.head(2)

,sentence,review_index,rating,sentence_lemma,word_count
0,“فندق راقي وسوف تتكرر زيارتي له”.,0,4,فندق راقي سوف تتكرر زيارتي له,6
2,لاشئ بصراحه الا ملاحظه واحدة اني شاهدة خنفسانا...,0,4,لاشئ بصراحه الا ملاحظه احده اني شاهده خنفسانا ...,26


In [ ]:
import json
UNIQUE_ASPECTS_PATH = f'{DATA_PATH}/fine_aspects.txt'
with open(UNIQUE_ASPECTS_PATH, 'r') as f:
    unique_aspects = json.loads(f.read())

unique_aspects

['FACILITIES#CLEANLINESS',
 'FACILITIES#COMFORT',
 'FACILITIES#DESIGN_FEATURES',
 'FACILITIES#GENERAL',
 'FACILITIES#MISCELLANEOUS',
 'FACILITIES#PRICES',
 'FACILITIES#QUALITY',
 'FOOD_DRINKS#MISCELLANEOUS',
 'FOOD_DRINKS#PRICES',
 'FOOD_DRINKS#QUALITY',
 'FOOD_DRINKS#STYLE_OPTIONS',
 'HOTEL#CLEANLINESS',
 'HOTEL#COMFORT',
 'HOTEL#DESIGN_FEATURES',
 'HOTEL#GENERAL',
 'HOTEL#MISCELLANEOUS',
 'HOTEL#PRICES',
 'HOTEL#QUALITY',
 'LOCATION#GENERAL',
 'ROOMS#CLEANLINESS',
 'ROOMS#COMFORT',
 'ROOMS#DESIGN_FEATURES',
 'ROOMS#GENERAL',
 'ROOMS#MISCELLANEOUS',
 'ROOMS#PRICES',
 'ROOMS#QUALITY',
 'ROOMS_AMENITIES#CLEANLINESS',
 'ROOMS_AMENITIES#COMFORT',
 'ROOMS_AMENITIES#DESIGN_FEATURES',
 'ROOMS_AMENITIES#GENERAL',
 'ROOMS_AMENITIES#MISCELLANEOUS',
 'ROOMS_AMENITIES#PRICES',
 'ROOMS_AMENITIES#QUALITY',
 'SERVICE#GENERAL']

In [ ]:
# features = 'sentence'
features_lemma = 'text_lemma'
# features_lemma = 'combined_features'
target = 'fine_aspects'
classes = unique_aspects

In [ ]:
# import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, roc_auc_score
from sklearn.metrics import confusion_matrix as cm, classification_report as cr

metrics_dict =  {'Accuracy': accuracy_score,
                 'Precision': precision_score,
                 'Recall': recall_score,
                 'F1-score': f1_score,
                 'Logloss': log_loss,
                 'AUC': roc_auc_score}

In [ ]:
# classes = sorted(df[target].unique())
# def pretty_cm(y_true, y_pred, classes=classes, normalized=True):
#     # if y_true is one-hot-encoded --> collapse it 
#     y_true = y_true.argmax(axis=1) if len(y_true.shape) > 1 else y_true
    
#     confusion_mtx = cm(y_true, y_pred)
#     cm_df = pd.DataFrame(confusion_mtx, index=classes, columns=classes)
    
#     if normalized:
#         cm_df = cm_df.apply(lambda row: row/row.sum(), axis=1).round(4)
#     return cm_df

In [ ]:
def pretty_cr(y_true, y_pred, classes=classes):
    # if y_true is one-hot-encoded --> collapse it 
    y_true = y_true.argmax(axis=1) if len(y_true.shape) > 1 else y_true
    report = cr(y_true, y_pred, output_dict=True, target_names=classes)
    cr_df = pd.DataFrame(report).transpose()
    
    # support is just the number of samples in each class
    cr_df['support'] = cr_df['support'].astype('int')
    
    # accuracy has no micro or macro averaging!
    cr_df.loc['accuracy', ['precision', 'recall']] = ""
    cr_df.loc['accuracy', ['support']] = len(y_true)
    
    # remove the accuracy row to the end
    acc_row = cr_df.loc['accuracy']
    cr_df.drop('accuracy', axis=0, inplace=True)
    cr_df = cr_df.append(acc_row)
    
    return cr_df

## Predict latest trained student so far 

In [ ]:
MODEL_PATH

'/content/drive/My Drive/projects/zeroshot-ar/models/bert-multilabel-semEval-and-hard'

In [ ]:
def raw_class_predict(predictor, raw_test_data, thresh=0.5, drop_thresh=0.0, return_df=True, preds_as_list=True, drop_one_hot_preds=False):
    preds_proba = pd.DataFrame(predictor.predict_proba(raw_test_data), columns=classes)
    # replace rows under the drop_threshh with nans
    preds_proba = preds_proba.apply(lambda row: row if row.max() >= drop_thresh else row.apply(lambda x: np.nan), axis=1)
    # get the indices of those rows
    # r = np.asarray(preds_proba).nonzero()[0]
    r, _ = np.where(preds_proba.isna())
    drop_indices = list(set(r))
    # print(drop_indices)
    preds_proba = preds_proba.drop(drop_indices).reset_index(drop=True)
    # return the filtered raw test data
    raw_test_data = pd.DataFrame(raw_test_data).drop(drop_indices).reset_index(drop=True).values

    preds_classes = preds_proba.apply(lambda row: row.apply(lambda x: 1 if x > thresh else 0), axis=1).values

    out_value = preds_classes

    if return_df:
        preds_df = pd.DataFrame(preds_proba, columns=classes).assign(Doc = raw_test_data).loc[:, ['Doc'] + classes]
        out_value = preds_df

    if preds_as_list:
        preds_df['Predicted Aspects'] = preds_df.apply(lambda row: row[classes].loc[row[classes] > thresh].index.tolist(), axis=1).tolist()
        # preds_df.drop(classes, axis=1, inplace=True)
        preds_df = preds_df.loc[:, ['Doc', 'Predicted Aspects'] + classes]
        out_value = preds_df
    
    if drop_one_hot_preds:
        preds_df.drop(classes, axis=1, inplace=True)
        out_value = preds_df

    # return preds_proba, drop_indices
    return out_value

***Load The teacher and the student predictors***

In [ ]:
##############################Student########################
STUDENT_MODEL_PATH ="/content/drive/My Drive/projects/zeroshot-ar/models/bert-multilabel-semEval-and-hard"
##############################Teacher########################
TEACHER_MODEL_PATH = '/content/drive/My Drive/projects/zeroshot-ar/models/bert-multilabel-semEval'

***load the model predictor***

In [ ]:
predictor_student  = ktrain.load_predictor(f'{STUDENT_MODEL_PATH}/predictor.hd5')

In [ ]:
predictor_teacher  = ktrain.load_predictor(f'{TEACHER_MODEL_PATH}/predictor.hd5')


In [ ]:
# docs = df['sentence_lemma']#.iloc[:100000]; docs.tolist()[:10]

In [ ]:
# predictor.predict(docs.values)

In [ ]:
# np.array([c[0] for c in predictor.predict(doc)]) == np.array(classes)

***Generate annotations, based on the threshold score of each class. ( 0>= `thresh` >= 1)***

In [ ]:
%%time
if False:
    student_df = raw_class_predict(predictor_student, df['sentence_lemma'].values, thresh=0.4, drop_thresh=0.0, preds_as_list=True)
    student_df.to_json(f'{DATA_PATH}/VOTING/preds_aspect_student_01.json')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [ ]:
student_preds_df= pd.read_json(f'{DATA_PATH}/VOTING/preds_aspect_student_01.json')

In [ ]:
%%time
if False:
    teacher_preds_df = raw_class_predict(predictor_teacher, df['sentence_lemma'].values, thresh=0.4, drop_thresh=0.0, preds_as_list=True)
    teacher_preds_df.to_json(f'{DATA_PATH}/VOTING/preds_aspect_teacher_01.json')

CPU times: user 56min 51s, sys: 5min 28s, total: 1h 2min 20s
Wall time: 1h 39min 17s


In [ ]:
teacher_preds_df= pd.read_json(f'{DATA_PATH}/VOTING/preds_aspect_teacher_01.json')

***Ensemble the probs of each model by calculating the averege score***

In [ ]:
assert ( (teacher_preds_df['Doc'] == student_preds_df['Doc']).sum() == len(student_preds_df) == len(teacher_preds_df) ) 

In [ ]:
teacher_preds_df.drop_duplicates(subset=['Doc'] ,inplace=True)
teacher_preds_df.sort_values(by=['Doc'] , inplace = True)


student_preds_df.drop_duplicates(subset=['Doc'],inplace=True )
student_preds_df.sort_values(by=['Doc'] , inplace = True)


# Check if the two frames identically sorted 
assert len(teacher_preds_df) == len(student_preds_df)
assert teacher_preds_df['Doc'].tolist() == student_preds_df['Doc'].tolist()

In [ ]:
student_preds_df[classes] = ((student_preds_df[classes].to_numpy()  + teacher_preds_df[classes].to_numpy()) / 2 )

In [ ]:
student_preds_df['Predicted Aspects'] = student_preds_df.apply(lambda row: row[classes].loc[row[classes] > 0.4].index.tolist() , axis=1)

***Remove records without aspects***

In [ ]:
student_preds_df = student_preds_df[~(student_preds_df['sentence aspect'].apply(lambda x : len(x) )==0)]

## Create the Combined Data Set

In [ ]:
student_preds_df.rename(columns={'Doc': 'text_lemma', 'Predicted Aspects': 'fine_aspects'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_combined = pd.concat([student_preds_df, sem_eval[sem_eval['split'] == 'Train']], ignore_index=True, join='inner')

In [ ]:
df_combined.head(20)

,text_lemma,fine_aspects
0,ايضا يفضل ادخال القنوات الرياضيه التي تنقل ا...,[]
1,عملوا لي up grade سكنت في الجناح الكبير الدخو...,"[FACILITIES#GENERAL, ROOMS#DESIGN_FEATURES]"
2,لكن بحتاج لتغيير في ارضيات الغرف لغيير المدخن...,[ROOMS#DESIGN_FEATURES]
3,0 0,[HOTEL#GENERAL]
4,0 1,[]
5,0 10,[HOTEL#GENERAL]
6,0 100,[HOTEL#PRICES]
7,0 5,[]
8,0 الارتفاع اكثر من المذكور,"[HOTEL#PRICES, HOTEL#QUALITY]"
9,0 الاستقبال غير محترمين خاصه المدير المصري عام...,[SERVICE#GENERAL]


In [ ]:
COMBINED_DATA_FILE = f'{DATA_PATH}/combined_data_for_teacher_student_voting.json'

In [ ]:
df_combined.to_json(COMBINED_DATA_FILE)

In [ ]:
pd.read_json(COMBINED_DATA_FILE).equals(df_combined)

True

In [ ]:
ex = sem_eval[sem_eval['text_lemma'] == 'الكثير من الشاي او القهوه مع الكعك منزليا كعك اضافيه مما كان لمسه رائعه']; ex

,text,terms,fine_aspects,polarity,split,coarse_aspects,text_lemma,word_count
4799,الكثير من الشاي أو القهوة مع الكعك منزليا وكعك...,"[الشاي, لقهوة, الكعك, كعك]","[FOOD_DRINKS#QUALITY, FOOD_DRINKS#QUALITY, FOO...","[positive, positive, positive, positive]",Train,[FOOD_DRINKS],الكثير من الشاي او القهوه مع الكعك منزليا كعك ...,14


## Train the Student Model on the Combined Dataset

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the multi-labels into arrays
mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(labeled_df[['aspects', 'polarity']].apply(lambda row: row['aspects'] + [row['polarity']], axis=1))
y = mlb.fit_transform(df_combined[target])
y.shape

(636097, 34)

In [ ]:
def multilabel_sample(y, size=1000, min_count=5, seed=None):
    """ Takes a matrix of binary labels `y` and returns
        the indices for a sample of size `size` if
        `size` > 1 or `size` * len(y) if size =< 1.

        The sample is guaranteed to have > `min_count` of
        each label.
    """
    try:
        if (np.unique(y).astype(int) != np.array([0, 1])).all():
            raise ValueError()
    except (TypeError, ValueError):
        raise ValueError('multilabel_sample only works with binary indicator matrices')

    if (y.sum(axis=0) < min_count).any():
        raise ValueError('Some classes do not have enough examples. Change min_count if necessary.')

    if size <= 1:
        size = np.floor(y.shape[0] * size)

    if y.shape[1] * min_count > size:
        msg = "Size less than number of columns * min_count, returning {} items instead of {}."
        warnings.warn(msg.format(y.shape[1] * min_count, size))
        size = y.shape[1] * min_count

    rng = np.random.RandomState(seed if seed is not None else np.random.randint(1))

    if isinstance(y, pd.DataFrame):
        choices = y.index
        y = y.values
    else:
        choices = np.arange(y.shape[0])

    sample_idxs = np.array([], dtype=choices.dtype)

    # first, guarantee > min_count of each label
    for j in range(y.shape[1]):
        label_choices = choices[y[:, j] == 1]
        label_idxs_sampled = rng.choice(label_choices, size=min_count, replace=False)
        sample_idxs = np.concatenate([label_idxs_sampled, sample_idxs])

    sample_idxs = np.unique(sample_idxs)

    # now that we have at least min_count of each, we can just random sample
    sample_count = int(size - sample_idxs.shape[0])

    # get sample_count indices from remaining choices
    remaining_choices = np.setdiff1d(choices, sample_idxs)
    remaining_sampled = rng.choice(remaining_choices,
                                   size=sample_count,
                                   replace=False)

    return np.concatenate([sample_idxs, remaining_sampled])


def multilabel_sample_dataframe(df, labels, size, min_count=5, seed=None):
    """ Takes a dataframe `df` and returns a sample of size `size` where all
        classes in the binary matrix `labels` are represented at
        least `min_count` times.
    """
    idxs = multilabel_sample(labels, size=size, min_count=min_count, seed=seed)
    return df.loc[idxs]


def multilabel_train_test_split(X, Y, size, min_count=5, seed=None):
    """ Takes a features matrix `X` and a label matrix `Y` and
        returns (X_train, X_test, Y_train, Y_test) where all
        classes in Y are represented at least `min_count` times.
    """
    index = Y.index if isinstance(Y, pd.DataFrame) else np.arange(Y.shape[0])

    test_set_idxs = multilabel_sample(Y, size=size, min_count=min_count, seed=seed)
    train_set_idxs = np.setdiff1d(index, test_set_idxs)

    test_set_mask = index.isin(test_set_idxs)
    train_set_mask = ~test_set_mask

    return (X[train_set_mask], X[test_set_mask], Y[train_set_mask], Y[test_set_mask])

In [ ]:
# X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df[features_lemma].values, 
#                                                                       df[target].values,
#                                                                       test_size=0.1,
#                                                                       val_size=0.1,
#                                                                       random_state=1,
#                                                                       stratify=df[target])

X_train, X_val, y_train, y_val = multilabel_train_test_split(df_combined[features_lemma].values, 
                                                             pd.DataFrame(y), 
                                                             size=0.0025, 
                                                             min_count=2,
                                                             seed=0)

In [ ]:
y_train, y_val = y_train.values, y_val.values

In [ ]:
y_train.shape, y_val.shape

((634507, 34), (1590, 34))

In [ ]:
(df_combined[features_lemma].apply(lambda review: len(review.split())).quantile(1.0))  

86.0

In [ ]:
MODEL_NAME = 'aubmindlab/bert-base-arabertv02'
# MODEL_NAME = 'bashar-talafha/multi-dialect-bert-base-arabic'
MAX_LEN = 86
BS = 32

In [ ]:
preproc = text.Transformer(MODEL_NAME, maxlen=MAX_LEN, batch_size=BS,
                           class_names=mlb.classes_)

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

In [ ]:
MAX_FEATURES = preproc.get_tokenizer().vocab_size
MAX_FEATURES

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

64000

In [ ]:
%%time
trn = preproc.preprocess_train(X_train, y_train)

preprocessing train...
language: ar
train sequence lengths:
	mean : 11
	95percentile : 34
	99percentile : 52


Is Multi-Label? True
CPU times: user 1min 22s, sys: 2.13 s, total: 1min 24s
Wall time: 1min 24s


In [ ]:
val = preproc.preprocess_test(X_val, y_val)

preprocessing test...
language: ar
test sequence lengths:
	mean : 11
	95percentile : 34
	99percentile : 56


#### Extracting the Classifyer

Transformer models are multitask learners. We just need the classifier part of it, so we extract it and store it in a variable `model`

Then To use ktrain, one simply wraps the `model` and the data in a `Learner` object using the `get_learner` function. This Learner object will be used to help tune and train our network.

In [ ]:
model = preproc.get_classifier()

Downloading:   0%|          | 0.00/742M [00:00<?, ?B/s]

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  135193344 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  26146     
                                                                 
Total params: 135,219,490
Trainable params: 135,219,490
Non-trainable params: 0
_________________________________________________________________


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=BS)

In [ ]:
# # rebuild the model to train from scratch 
# rebuild the model to train from scratch 
# learner.set_model(preproc.get_classifier())
learner.set_weight_decay(0.005)

/usr/local/lib/python3.7/dist-packages/ktrain/core.py:374: UserWarning:

recompiling model to use AdamWeightDecay as opimizer with weight decay of 0.005



In [ ]:
MODEL_PATH

'/content/drive/My Drive/projects/zeroshot-ar/models/bert-multilabel-semEval-and-hard'

In [ ]:
%%time
learner.autofit(1e-5, epochs=1, checkpoint_folder=MODEL_PATH+'/Votes_Student')



begin training using triangular learning rate policy with max lr of 1e-05...
19829/19829 [==============================] - 7455s 369ms/step - loss: 0.0618 - accuracy: 0.6131 - val_loss: 0.0188 - val_accuracy: 0.7233
CPU times: user 1h 28min 18s, sys: 12min 23s, total: 1h 40min 41s
Wall time: 2h 4min 14s


In [ ]:
learner.model.load_weights(f'{MODEL_PATH}/weights-{1:02d}.hdf5')

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=preproc)

In [ ]:
predictor.save(f'{MODEL_PATH}/Votes_Student/predictor.hd5')

In [ ]:
predictor = ktrain.load_predictor(f'{MODEL_PATH}/predictor.hd5')

## Testing on the Gold Standard Test Data

In [ ]:
def class_predict(learner, test_data, raw_test_data, thresh=0.5, drop_thresh=0.0, return_proba=False):
    preds_proba = pd.DataFrame(learner.predict(test_data), columns=mlb.classes_)
    # replace rows under the drop_threshh with nans
    preds_proba = preds_proba.apply(lambda row: row if row.max() >= drop_thresh else row.apply(lambda x: np.nan), axis=1)
    # get the indices of those rows
    # r = np.asarray(preds_proba).nonzero()[0]
    r, _ = np.where(preds_proba.isna())
    drop_indices = list(set(r))
    # print(drop_indices)
    preds_proba = preds_proba.drop(drop_indices).reset_index(drop=True)
    # return the filtered raw test data
    raw_test_data = pd.DataFrame(raw_test_data).drop(drop_indices).reset_index(drop=True).values

    preds_classes = preds_proba.apply(lambda row: row.apply(lambda x: 1 if x > thresh else 0), axis=1).values
    # return preds_proba, drop_indices
    if return_proba:
        return preds_proba, raw_test_data
    else:
        return preds_classes, raw_test_data 


In [ ]:
y_test = mlb.fit_transform(sem_eval.loc[sem_eval['split'] == 'Test', target])
print("This is the target aspects after transformation:\n")
print(y_test)
print(f"Target Shape: {y_test.shape}")
X_test = sem_eval.loc[sem_eval['split'] == 'Test', features_lemma].values


This is the target aspects after transformation:

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]
Target Shape: (1227, 34)


In [ ]:
test = preproc.preprocess_test(X_test, y_test)

preprocessing test...
language: ar
test sequence lengths:
	mean : 20
	95percentile : 50
	99percentile : 64


In [ ]:
from sklearn.metrics import log_loss

In [ ]:
%%time
preds_test, y_test_filtered = class_predict(learner, test, y_test, thresh=0.5, drop_thresh=0.0)

CPU times: user 2.71 s, sys: 351 ms, total: 3.06 s
Wall time: 5.51 s


In [ ]:
len(y_test), len(y_test_filtered), len(preds_test)

(1227, 1227, 1227)

In [ ]:
# this is the new prediction
f1_score(y_test_filtered, preds_test, average='micro')

0.6830357142857143

### Correlation Between Class-based F1 Score and its Support (Count) in the Training Data

In [ ]:
def by_class_res(y_true, preds, plot=True, cor=False, model_name=None):
    res = (
        df_combined.loc[:, 'fine_aspects']
        .explode()
        .value_counts()
        .reset_index()
        # .pipe(lambda df: df.assign(index = df['index'].str.lower()))
        .sort_values('index')
    )

    res['f1-score'] = f1_score(y_true, preds, average=None)

    res = res.sort_values('fine_aspects', ascending=True)

    if cor:
        res = res[['fine_aspects', 'f1-score']].corr().loc['fine_aspects', ['f1-score']].values

    if plot and not cor:
        subfig = make_subplots(specs=[[{"secondary_y": True}]])
        fig1 = (
            res
             .set_index('index')
             .pipe(
                 lambda df: px.bar(
                     df, x=df.index, y='fine_aspects',
                 )
             )
        )

        fig2 = (
            px.scatter(
                res, x='index', y='f1-score',
            ).update_traces(
                mode='markers+lines',yaxis='y2',
                marker=dict(color='grey')
            )
        )
        subfig.add_traces(fig1.data + fig2.data)
        subfig.layout.yaxis.title="Aspect Category Count"
        subfig.layout.yaxis2.title="F1 Score"
        subfig.data[1].name = 'F1 Score'
        subfig.data[1].showlegend = True
        subfig.data[0].name = 'Aspect Category'
        subfig.data[0].showlegend = True
        subfig.update_layout(legend=dict(orientation='h', y=1.05, x=0.35))
        subfig.for_each_trace(lambda t: t.update(marker=dict(color=t.marker.color)))
        plot_title = "Correlation Between Class Count and its F1 Score"
        if model_name:
            plot_title = f"{plot_title} ({model_name})"
        subfig.update_layout(title=plot_title, height=600)


        res = subfig

    return res

pearson correlation:

In [ ]:
by_class_res(y_test_filtered, preds_test, cor=True).round(2)

array([0.72])

Detailed result per class:

In [ ]:
byclass_f1_arabert_ensemble_df = by_class_res(y_test_filtered, preds_test, cor=False, plot=False).round(4)
byclass_f1_arabert_ensemble_df.sort_values('f1-score', ascending=False)

,index,fine_aspects,f1-score
0,SERVICE#GENERAL,210855,0.9077
6,FOOD_DRINKS#QUALITY,46402,0.8424
1,LOCATION#GENERAL,145702,0.8333
7,ROOMS#CLEANLINESS,41390,0.7630
8,ROOMS#DESIGN_FEATURES,40282,0.7162
2,HOTEL#GENERAL,137709,0.7126
4,HOTEL#CLEANLINESS,63021,0.6950
12,FOOD_DRINKS#STYLE_OPTIONS,16179,0.6506
5,HOTEL#PRICES,51479,0.6358
3,FACILITIES#GENERAL,119534,0.5249


In [ ]:
# byclass_f1_arabert_student_df.to_csv(f'{RES_PATH}/byclass_f1_arabert_hard_student.csv', index=False)

In [ ]:
teacher_f1 = pd.read_csv(f'{RES_PATH}/byclass_f1_arabert_teacher.csv')
byclass_f1_arabert_student_df = pd.read_csv(f'{RES_PATH}/byclass_f1_arabert_student.csv')

In [ ]:
byclass_f1_arabert_student_df

,index,fine_aspects,f1-score
0,ROOMS_AMENITIES#PRICES,13,0.0000
1,ROOMS#MISCELLANEOUS,30,0.0000
2,ROOMS_AMENITIES#COMFORT,35,0.0000
3,FACILITIES#COMFORT,47,0.0000
4,FOOD_DRINKS#MISCELLANEOUS,51,0.0000
5,ROOMS_AMENITIES#DESIGN_FEATURES,54,0.0000
6,ROOMS_AMENITIES#MISCELLANEOUS,55,0.0000
7,FACILITIES#MISCELLANEOUS,70,0.0000
8,FACILITIES#QUALITY,159,0.0000
9,FOOD_DRINKS#PRICES,195,0.0000


In [ ]:
byclass_f1_arabert_ensemble_df_both = byclass_f1_arabert_ensemble_df.merge(teacher_f1, on='index').rename(
    columns={
        'index': 'Aspect Category',
        'fine_aspects_x': 'Category Count in the Combined Data',
        'f1-score_x': 'f1-score of Student',
        'fine_aspects_y': 'Category Count in the SemEval Data',
        'f1-score_y': 'f1-score of Teacher',
    }
)

In [ ]:
byclass_f1_arabert_ensemble_df_both

,Aspect Category,Category Count in the Combined Data,f1-score of Student,Category Count in the SemEval Data,f1-score of Teacher
0,ROOMS_AMENITIES#PRICES,13,0.0000,13,0.0000
1,ROOMS#MISCELLANEOUS,30,0.0000,30,0.0000
2,ROOMS_AMENITIES#COMFORT,33,0.0000,33,0.0000
3,FOOD_DRINKS#PRICES,41,0.0000,41,0.0000
4,FACILITIES#COMFORT,47,0.0000,47,0.0000
5,FOOD_DRINKS#MISCELLANEOUS,51,0.0000,51,0.0000
6,ROOMS_AMENITIES#DESIGN_FEATURES,54,0.0000,54,0.0000
7,ROOMS_AMENITIES#MISCELLANEOUS,55,0.0000,55,0.0000
8,FACILITIES#MISCELLANEOUS,70,0.0000,70,0.0000
9,FACILITIES#QUALITY,136,0.0000,136,0.0000


In [ ]:
byclass_f1_arabert_ensemble_df_both.to_csv(f'{RES_PATH}/byclass_f1_arabert_ensemble.csv', index=False)

## Ensembling on Teacher and Student Models

In [ ]:
teacher_learner = ktrain.get_learner(predictor_teacher.model, train_data=trn, val_data=test)

In [ ]:
preds_test_proba_teacher, y_test_filtered_teacher = class_predict(teacher_learner, test, y_test, thresh=0.5, drop_thresh=0.0, return_proba=True)

In [ ]:
preds_test_proba, y_test_filtered = class_predict(learner, test, y_test, thresh=0.5, drop_thresh=0.0, return_proba=True)

In [ ]:
ensemble_preds_proba = (preds_test_proba_teacher*0.3 + preds_test_proba*0.5)

In [ ]:
preds_classes = ensemble_preds_proba.apply(lambda row: row.apply(lambda x: 1 if x > 0.5 else 0), axis=1)

In [ ]:
f1_score(y_test_filtered, preds_classes, average='micro')

0.6905154153918569

### 

`preds_aspect_07_04`: f1: 0.6765 (testing thresh=0.4) - 1 epoch  

`preds_aspect_06_04`: f1: 0.6762 (testing thresh=0.6) - 1 epoch

`preds_aspect_05_04`: f1: 0.67919, loss: 3.445 (testing thresh=0.5) - 1 epoch (best till now)

`preds_aspect_04_04`: f1: 0.67933, loss: 4.3506 (testing thresh=0.6) - 1 epoch (best till now)

`preds_aspect_08_08`: f1:0.6830, -1 epoch

